In [2]:
# Importation des packages nécessaires
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import openpyxl

# Chargement du fichier Excel contenant la liste des employés
df_employes = pd.read_excel('employees.xlsx')
df_employes

,ID,Nom
0,1,AVLESSI Kokou Dieudonné
1,2,AYIKOE Abra Odile
2,3,LABTECH
3,3,NANA Kossi Mawuto
4,4,TSEKU Akouvi Mawussé
5,5,DOGBASSEY Debora
6,6,JIMONGOU Kountoléne Geneviève
7,7,AWOKOU Komivi
8,8,BATIGMA Ousséou Samiratou
9,9,BIRAMAH Aïcha Kalé


In [3]:
# Importation des packages nécessaires
import pandas as pd
import numpy as np
from datetime import datetime, timedelta               
import random
import openpyxl

# Chargement du fichier Excel contenant la liste des employés
df_employes = pd.read_excel('employees.xlsx')

# Demande de la date de départ pour l'élaboration du planning
date_depart = input("Veuillez entrer la date de départ pour l'élaboration du planning (format: JJ/MM/AAAA) : ")
date_depart = datetime.strptime(date_depart, "%d/%m/%Y")

# Demande du nombre de shifts à planifier
nb_shifts = int(input("Veuillez entrer le nombre de shifts à planifier : "))

# Demande du pourcentage des employés à planifier le matin pour chaque shift
pourcentages_matin = []
for i in range(nb_shifts):
    pourcentage = float(input(f"Veuillez entrer le pourcentage d'employés à planifier le matin pour le shift {i+1} : "))
    pourcentages_matin .append(pourcentage)

# Demande du nombre de jours de repos à accorder pour chaque jour
jours_semaine = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi", "Samedi", "Dimanche"]
jours_repos = []
for jour in jours_semaine:
    nb_repos = int(input(f"Veuillez entrer le nombre de jours de repos à accorder pour {jour} : "))
    jours_repos.append(nb_repos)

# Définition des moments prédéfinis pour les pauses
moments_pauses = ["9h30-10h00", "10h30-11h00", "11h30-12h00", "12h30-13h00", "16h30-17h00", "17h30-18h00", "18h30-19h00", "19h30-20h00"]
 
# Calcul du nombre d'employés à planifier pour chaque shift
nb_employes_total = len(df_employes)
nb_employes_matin = [int(pourcentage * nb_employes_total / 100) for pourcentage in pourcentages_matin]
nb_employes_soir = [nb_employes_total - nb for nb in nb_employes_matin]

# Vérification si le nombre d'employés à planifier est suffisant
if sum(nb_employes_matin) > nb_employes_total or sum(nb_employes_soir) > nb_employes_total:
    print("Erreur : Le nombre total d'employés à planifier dépasse le nombre d'employés disponibles.")
    exit()

# Fonction pour générer un planning pour un shift donné
def generer_planning_shift(shift, nb_employes, date):
    planning_shift = pd.DataFrame(columns=["ID", "Nom", "Date", "Shift"])
    employes_disponibles = df_employes.copy()
    
    for _ in range(nb_employes):
        # Sélection aléatoire d'un employé disponible
        index = random.randint(0, len(employes_disponibles)-1)
        employe = employes_disponibles.iloc[index]
        employes_disponibles.drop(index, inplace=True)
        
        # Ajout de l'employé au planning du shift
        row = {
            "ID": employe["ID"],
            "Nom": employe["Nom"],
            "Date": date.strftime("%d/%m/%Y"),
            "Shift": shift
        }
        planning_shift = planning_shift.append(row, ignore_index=True)
        
    return planning_shift

# Création du planning pour chaque jour en respectant les contraintes
planning_total = pd.DataFrame(columns=["ID", "Nom", "Date", "Shift"])

for i in range(nb_shifts):
    # Calcul de la date pour le premier shift
    date_shift = date_depart + timedelta(days=i*14)
    
    for j, jour in enumerate(jours_semaine):
        # Vérification des contraintes pour le repos des employés
        if j > 0:
            jours_travail_avant_repos = sum(jours_repos[:j])
            if jours_travail_avant_repos < 4 or jours_travail_avant_repos > 7:
                # Forcer un jour de repos si les contraintes ne sont pas respectées
                jours_repos[j] += 1
        
        # Génération du planning pour le shift du matin
        planning_matin = generer_planning_shift("Matin", nb_employes_matin[i], date_shift)
        
        # Génération du planning pour le shift du soir
        planning_soir = generer_planning_shift("Soir", nb_employes_soir[i], date_shift)
        
        # Ajout des pauses au planning
        planning_pauses = pd.DataFrame(columns=["ID", "Nom", "Date", "Shift"])
        
        for moment_pause in moments_pauses:
            row = {
                "ID": "",
                "Nom": "Pause",
                "Date": date_shift.strftime("%d/%m/%Y"),
                "Shift": moment_pause
            }
            planning_pauses = planning_pauses.append(row, ignore_index=True)
        
        # Concaténation des plannings du matin, du soir et des pauses
        planning_jour = pd.concat([planning_matin, planning_soir, planning_pauses])
        
        # Ajout du planning du jour au planning total
        planning_total = pd.concat([planning_total, planning_jour])
        
        # Mise à jour de la date pour le prochain jour
        date_shift += timedelta(days=1)

# Sauvegarde du planning en fichier Excel
planning_total.to_excel("planning.xlsx", index=False)


Veuillez entrer la date de départ pour l'élaboration du planning (format: JJ/MM/AAAA) : 22/05/2023
Veuillez entrer le nombre de shifts à planifier : 2
Veuillez entrer le pourcentage d'employés à planifier le matin pour le shift 1 : 51
Veuillez entrer le pourcentage d'employés à planifier le matin pour le shift 2 : 49
Veuillez entrer le nombre de jours de repos à accorder pour Lundi : 4
Veuillez entrer le nombre de jours de repos à accorder pour Mardi : 4
Veuillez entrer le nombre de jours de repos à accorder pour Mercredi : 4
Veuillez entrer le nombre de jours de repos à accorder pour Jeudi : 4
Veuillez entrer le nombre de jours de repos à accorder pour Vendredi : 4
Veuillez entrer le nombre de jours de repos à accorder pour Samedi : 4
Veuillez entrer le nombre de jours de repos à accorder pour Dimanche : 0
Erreur : Le nombre total d'employés à planifier dépasse le nombre d'employés disponibles.


C:\Users\atchaneyou\AppData\Local\Temp\ipykernel_11772\3636311986.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  planning_shift = planning_shift.append(row, ignore_index=True)
C:\Users\atchaneyou\AppData\Local\Temp\ipykernel_11772\3636311986.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  planning_shift = planning_shift.append(row, ignore_index=True)
C:\Users\atchaneyou\AppData\Local\Temp\ipykernel_11772\3636311986.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  planning_shift = planning_shift.append(row, ignore_index=True)


KeyError: '[13] not found in axis'

In [1]:
import pandas as pd
import numpy as np

# Importation des données des employés à partir du fichier Excel
employees_data = pd.read_excel("employees.xlsx", usecols=["ID", "Nom"])
# Chargement du fichier Excel contenant la liste des employés
#df_employes = pd.read_excel('employees.xlsx')

# Demande de la date de départ pour l'élaboration du planning
start_date = input("Veuillez entrer la date de départ pour l'élaboration du planning (format : AAAA-MM-JJ) : ")

# Demande du nombre de shift à planifier
num_shifts = int(input("Veuillez entrer le nombre de shift à planifier : "))

# Demande du pourcentage d'employés à planifier le matin
percent_morning_shift = int(input("Veuillez entrer le pourcentage d'employés à planifier le matin : "))

# Demande du nombre de repos à accorder pour chaque jour
num_repos_per_day = int(input("Veuillez entrer le nombre de repos à accorder pour chaque jour : "))

# Définition des plages horaires pour les pauses
pause_morning_start = "9:30"
pause_morning_end = "13:00"
pause_afternoon_start = "16:30"
pause_afternoon_end = "20:00"

# Calcul du nombre d'employés à planifier pour chaque shift
num_employees = len(employees_data)
num_morning_shift = int(num_employees * percent_morning_shift / 100)
num_afternoon_shift = num_employees - num_morning_shift

# Création du tableau Excel pour le planning
planning = pd.DataFrame(columns=["Date", "Shift", "Employé"])

# Génération du planning pour chaque jour
for i in range(num_shifts):
    shift_date = pd.to_datetime(start_date) + pd.DateOffset(days=i)
    
    # Ajout des employés au planning du shift du matin
    morning_employees = np.random.choice(employees_data["Nom"], size=num_morning_shift, replace=False)
    morning_shift = pd.DataFrame({
        "Date": shift_date,
        "Shift": "Matin",
        "Employé": morning_employees
    })
    planning = pd.concat([planning, morning_shift])
    
    # Ajout des employés au planning du shift de l'après-midi
    afternoon_employees = employees_data[~employees_data["Nom"].isin(morning_employees)]
    afternoon_employees = afternoon_employees.sample(n=num_afternoon_shift)
    afternoon_shift = pd.DataFrame({
        "Date": shift_date,
        "Shift": "Après-midi",
        "Employé": afternoon_employees["Nom"]
    })
    planning = pd.concat([planning, afternoon_shift])
    
    # Ajout des repos pour chaque jour
    for j in range(num_repos_per_day):
        rest_date = pd.to_datetime(shift_date) + pd.DateOffset(days=j)
        rest_employees = np.random.choice(employees_data["Nom"], size=1, replace=False)
        rest_shift = pd.DataFrame({
            "Date": rest_date,
            "Shift": "Repos",
            "Employé": rest_employees
        })
        planning = pd.concat([planning, rest_shift])

# Exportation du planning vers un fichier Excel
planning.to_excel("planning.xlsx", index=False)

print("Le planning a été généré et enregistré dans le fichier 'planning.xlsx'.")


Veuillez entrer la date de départ pour l'élaboration du planning (format : AAAA-MM-JJ) : 2023-05-22
Veuillez entrer le nombre de shift à planifier : 2
Veuillez entrer le pourcentage d'employés à planifier le matin : 51
Veuillez entrer le nombre de repos à accorder pour chaque jour : 49
Le planning a été généré et enregistré dans le fichier 'planning.xlsx'.


In [4]:
import pandas as pd
import numpy as np

# Importation des données des employés à partir du fichier Excel
employees_data = pd.read_excel("employees.xlsx", usecols=["ID", "Nom"])

# Demande de la date de départ pour l'élaboration du planning
start_date = input("Veuillez entrer la date de départ pour l'élaboration du planning (format : AAAA-MM-JJ) : ")

# Demande du nombre de shift à planifier
num_shifts = int(input("Veuillez entrer le nombre de shift à planifier : "))

# Demande du pourcentage d'employés à planifier le matin
percent_morning_shift = int(input("Veuillez entrer le pourcentage d'employés à planifier le matin : "))

# Demande du nombre de pauses à attribuer aux employés pour chaque jour
num_pauses_per_day = int(input("Veuillez entrer le nombre de pauses à attribuer aux employés pour chaque jour : "))

# Définition des plages horaires pour les pauses
pause_morning_start = "9:30"
pause_morning_end = "13:00"
pause_afternoon_start = "16:30"
pause_afternoon_end = "20:00"

# Calcul du nombre d'employés à planifier pour chaque shift
num_employees = len(employees_data)
num_morning_shift = int(num_employees * percent_morning_shift / 100)
num_afternoon_shift = num_employees - num_morning_shift

# Création du tableau Excel pour le planning
planning = pd.DataFrame(columns=["Date", "Shift", "Employé"])

# Génération du planning pour chaque jour
for i in range(num_shifts):
    shift_date = pd.to_datetime(start_date) + pd.DateOffset(days=i)
    
    # Ajout des employés au planning du shift du matin
    morning_employees = np.random.choice(employees_data["Nom"], size=num_morning_shift, replace=False)
    morning_shift = pd.DataFrame({
        "Date": shift_date,
        "Shift": "Matin",
        "Employé": morning_employees
    })
    planning = pd.concat([planning, morning_shift])
    
    # Ajout des employés au planning du shift de l'après-midi
    afternoon_employees = employees_data[~employees_data["Nom"].isin(morning_employees)]
    afternoon_employees = afternoon_employees.sample(n=num_afternoon_shift)
    afternoon_shift = pd.DataFrame({
        "Date": shift_date,
        "Shift": "Après-midi",
        "Employé": afternoon_employees["Nom"]
    })
    planning = pd.concat([planning, afternoon_shift])
    
    # Ajout des pauses pour chaque jour
    for j in range(num_pauses_per_day):
        pause_date = pd.to_datetime(shift_date) + pd.DateOffset(days=j)
        
        # Demande du nombre d'employés pour cette pause
        num_employees_pause = int(input(f"Veuillez entrer le nombre d'employés pour la pause du {pause_date.strftime('%Y-%m-%d')} : "))
        
        # Ajout des employés à la pause
        pause_employees = np.random.choice(employees_data["Nom"], size=num_employees_pause, replace=False)
           pause_shift = pd.DataFrame({
            "Date": pause_date,
            "Shift": "Pause",
            "Employé": pause_employees
        })
        planning = pd.concat([planning, pause_shift])

# Exportation du planning vers un fichier Excel
planning.to_excel("planning.xlsx", index=False)

print("Le planning a été généré et enregistré dans le fichier 'planning.xlsx'.")


IndentationError: unexpected indent (2463444227.py, line 65)